# Demonstration of pyQ3 python wrapper for DEW-ENKI-EQ3 aqueous speciation calculations
**Important: before running any calculations you must have a compiled set of DEW species objects. If you are running this on your own machine you should run the import_DEW.ipynb notebook before running this one.**

First import the required packages:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyQ3

## Calculate speciation for a fluid with defined molalities
In the previous notebook it was demonstrated how the DATA0 database file can be constructed for use with manual EQ3 calculations. The pyQ3 module also provides a python interface to these calculations. To demonstrate this we will use the default system. To create a customised system, see the first tutorial.

Create a DEW System:

In [2]:
dew_system = pyQ3.System()

### Speciation without specifying pH or fO2:
The pH of a solution can be calculated by charge balance if all other chemical components are constrained. Some solutions will be insensitive to fO2, so we will also skip this to start with.

We must specify the molality of elements present in the solution, in terms of their basis species. Let's check what the basis species are:

In [3]:
print(dew_system.basis_species_names)

['H2O', 'H+', 'AG+', 'AL+3', 'AU+', 'BA+2', 'CA+2', 'CO2(AQ)', 'SIO2(AQ)', 'CL-', 'SO4-2', 'CD+2', 'N2(AQ)', 'CR+2', 'CS+', 'CU+', 'EU+2', 'F-', 'FE+2', 'H3PO4(AQ)', 'HG+2', 'K+', 'MG+2', 'MN+2', 'NA+', 'PB+2', 'SR+2', 'U+4', 'ZN+2', 'O2']


In [4]:
molalities = {'NA+': 0.1,
              'CL-': 0.1,
              'SIO2(AQ)':0.5}

We need to choose a temperature and pressure to run the calculation at:

In [5]:
T = 500.0 + 273.15 # in K
P = 8500.0 # in bar

Now we can generate the fluid:

In [6]:
dew_fluid = pyQ3.Fluid(dew_system, T, P, molalities=molalities)

When we created the dew_fluid object several things happened in the background:
1. a DATA0 file covering the pressure and temperature of the fluid was generated
2. The EQPT processor was run to generate the data1 file from DATA0
3. An EQ3 input file was generated using the constraints we set
4. EQ3 was run to do the speciation calculation
5. The EQ3 output file was read in

We can now look up different properties of the fluid, for example the speciation:

In [8]:
dew_fluid.aqeuous_species

,species,molality,log_g,activity
0,SIO2(AQ),2.347172e-01,0.0000,2.347000e-01
1,SI2O4(AQ),1.323791e-01,0.0000,1.324000e-01
2,NA+,9.578317e-02,-0.1556,6.694000e-02
3,CL-,9.566903e-02,-0.1556,6.686000e-02
4,NACL(AQ),4.200542e-03,0.0000,4.201000e-03
5,SI3O6(AQ),1.312946e-04,0.0000,1.313000e-04
6,HSIO3-,1.307279e-04,-0.1556,9.136000e-05
7,HCL(AQ),1.304248e-04,0.0000,1.304000e-04
8,H+,1.206938e-04,-0.1556,8.435000e-05
9,OH-,1.041005e-04,-0.1556,7.275000e-05


We could check that the input molalities were read correctly by checking the elemental molalities:

In [9]:
dew_fluid.elemental_comp

,element,ppm,molality
0,O,917984.1388,.5737616028E+02
1,H,113643.0641,.1127523208E+03
2,SI,14042.7500,.5000000000E+00
3,CL,3545.3000,.1000000000E+00
4,NA,2298.9770,.1000000000E+00


Since the pH will be calculated by charge balance if we don't explicitly set it, we can check the pH:

In [10]:
dew_fluid.pH

4.0739

What about fO2?

In [11]:
dew_fluid.fO2

-12.6

But we didn't set fO2, nor is it constrained by the calculation! Well, actually the code sets it to an abitrary value of -12.6.